In [2]:
import numpy as np
import data_import
import runner
import pandas as pd

In [3]:
f_base = runner.PROJ_BASE + 'assets/rigidbody2d/isaiah_ura/simple_example'

runner.run_rigidbody2dcli(
    f'{f_base}.xml',
    f'{f_base}.out'
)

pd_data = data_import.read_file_to_pd_dataframe(f"{f_base}.out")
pd_data

0/0.05
      -2
-3.14643
      -4
LCPOperatorPI: Converged in 2 iterations.


,N,Q,v0,Minv,M,ipopt_sol,policy_sol,policy_converged,ipopt_converged,f_name,...,eigenvalues,cond_num,cond_num_no_inf,Q_num_off_diag_entries,thetas,min_theta,rank,nullity,ipopt_delta_KE,ipopt_policy
0,"[[-1.0, -0.524404, 0.0], [0.0, -0.851469, 0.0]...","[[2.0, 0.524404, 0.45], [0.524404, 2.0, 0.5244...","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9, -1.78606, ...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]...","[0.354839, 1.04881, 1.64516]","[0.354839, 1.04881, 1.64516]",True,True,/home/isaiah/scisim/assets/rigidbody2d/isaiah_...,...,"[2.9999994260849503, 1.5500000000000003, 1.450...",2.068964,2.068964,6,"[58.37187051208563, 63.256316049597, 58.371870...",58.371871,3,0,0.000007,"[0, 1, 2]"


# Jacobi method...

In [4]:
sim_idx = 0
# Q = pd_data['Q'][sim_idx]
Q = np.array([
    [5, -1, 2],
    [3, 8, -2],
    [1, 1, 4],
])
N = pd_data['N'][sim_idx]
v0 = pd_data['v0'][sim_idx]
# b = 2 * N.T @ v0
b = np.array([12, -25, 6])

print(Q)
print(b)

# normalization...?
# for i in range(len(b)):
#     norm_factor = sum(Q[i,:])
#     Q[i,:] /= norm_factor
#     b[i] /= norm_factor

# print(Q)
# print(b)

# Initialize Markov Decision Process model
actions = (0, 1)  # actions (0=left, 1=right)
states = list(range(len(b)))  # states (tiles) ((collision #s))
rewards = b  # Direct rewards per state ((b))
gamma = 0.5  # discount factor
# Transition probabilities per state-action pair
probs = Q

# Set value iteration parameters
max_iter = 10000  # Maximum number of iterations
delta = 1e-400  # Error tolerance
V = np.zeros(len(b))  # Initialize values
pi = [1] * len(b)  # Initialize policy


# Start value iteration
for i in range(max_iter):
    max_diff = 0  # Initialize max difference
    V_new = np.zeros(len(b))
    for s in states:
        max_val = 0
        for a in actions:

            # Compute state value
            val = rewards[s] / probs[s,s]  # Get direct reward
            for s_next in states:
                if s_next == s:
                    continue
                val -= probs[s][s_next] * (
                    (1 / probs[s,s]) * V[s_next]
                )  # Add discounted downstream values

            # Store value best action so far
            # max_val = min(max_val, val)
            max_val = val

            # Update best policy
            # if V[s] < val:
            #     pi[s] = actions[a]  # Store action with highest value

        V_new[s] = max_val  # Update value with highest value

        # Update maximum difference
        max_diff = max(max_diff, abs(V[s] - V_new[s]))

    # Update value functions
    V = V_new

    # If diff smaller than threshold delta for all states, algorithm terminates
    if max_diff < delta:
        break

print(V)
print(Q @ V + b)

[[ 5 -1  2]
 [ 3  8 -2]
 [ 1  1  4]]
[ 12 -25   6]
[ 1. -3.  2.]
[ 24. -50.  12.]


# Value iteration/Gauss Seidel (btw, no chance they are the same...)

In [5]:
sim_idx = 0
Q = pd_data['Q'][sim_idx]
N = pd_data['N'][sim_idx]
v0 = pd_data['v0'][sim_idx]
b = 2 * N.T @ v0
# b = np.array([12, -25, 6])

# Set value iteration parameters
max_iter = 10000  # Maximum number of iterations
delta = 1e-4  # Error tolerance
V_old = np.zeros(len(b))  # Initialize values

# Start value iteration
for i in range(max_iter):
    print(V_old)
    V_new = np.zeros(len(b))
    for s in states:
        val = b[s]
        for s_next in states:
            if s_next < s:
                val -= V_new[s_next] * Q[s, s_next]
            elif s_next == s:
                continue
            elif s_next > s:
                val -= V_old[s_next] * Q[s, s_next]
            
        val /= Q[s,s]

        V_new[s] = val
    V_old = V_new.copy()

    print(np.linalg.norm(Q @ V_old - b))
    if np.linalg.norm(Q @ V_old - b) < delta:
        break

print(V_old)
print(Q @ V_old - b)

[0. 0. 0.]
1.5285275317254479
[-1.         -1.31101312 -1.43125311]
0.10709378533876085
[-0.33421779 -1.11030512 -1.63368015]
0.024001721839768314
[-0.34129774 -1.05537197 -1.64649074]
0.0037901686650468726
[-0.35281894 -1.04899212 -1.64557128]
0.00034737728166506953
[-0.35469863 -1.04874035 -1.64521437]
2.600059758271824e-05
[-0.35484495 -1.04879557 -1.64516697]
[-7.62628710e-06  2.48570075e-05  0.00000000e+00]


In [17]:
sim_idx = 0
Q = pd_data['Q'][sim_idx]
N = pd_data['N'][sim_idx]
v0 = pd_data['v0'][sim_idx]
b = -2 * N.T @ v0

num_actions = 2   # 2 actions: in or out
num_states = len(b)
v = np.ones(num_states)

for i in range(100):
    #M = [self.pool.apply(update, args=(c,PT,self.v))\
    #    for (c,PT) in zip(self.costs,self.PT)]

    Vs = np.empty((num_states,num_actions))
    # "in"
    Vs[:,1] = (Q @ v - 2 * v - b) / 2
    # "out"
    Vs[:,0] = (-1/2) * v
    assert((num_states,num_actions) == Vs.shape)

    v = -np.amin(Vs,axis=1)
    assert((num_states,) == v.shape)

    print(v)
    # print(np.linalg.norm(Q @ V_old - b))
    objective = np.linalg.norm(np.minimum(Q @ v - b, v))
    print(objective)
    if objective < delta:
        break

[0.512798   1.04881112 1.51280138]
0.32155461766592924
[0.38461932 1.04209891 1.60962345]
0.07496581175347876
[0.3645943  1.05032068 1.64022361]
0.019488094927946533
[0.3555535  1.04754786 1.64257347]
0.006554846140912439
[0.35575183 1.04930223 1.64533469]
0.0028467335401209668
[0.35467055 1.04852623 1.64483007]
0.0013754931748964654
[0.35498756 1.04894206 1.64527683]
0.000682771395380502
[0.35477801 1.0487418  1.64509647]
0.0003408757280199762
[0.3548711  1.04884403 1.64519613]
0.0001703855203342263
[0.35482187 1.0487935  1.64514838]
8.518733194340805e-05
